In [66]:
from urllib import request
import lxml
from lxml import html
import Utils as utils
import re
from data_amount import *

In [54]:
def stop(url):
    """
    This helper function will return true when the url
    contains '/data' at the end. 
    """
    return url[-5:] == '/data'

In [55]:
def stop_tests():
    """
    This function will test the stopping function. 
    """
    assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i/data'), True)
    assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i'), False)
    print("All Tests Passed!")
stop_tests()

All Tests Passed!


<ipython-input-55-9b4c3fab3d58>:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i/data'), True)
<ipython-input-55-9b4c3fab3d58>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i'), False)


In [56]:
def prune(url, browse_boolean):
    """
    This function will return true when the graph should continue searching
    and false when the graph should stop. 
    """
    url = url.lower().strip()
#     if 'data' not in url:
#         return False 
    if 'linkedin'in url:
        return False
    elif 'twitter' in url:
        return False
    elif 'socrata' in url:
        return False
    elif 'login' in url:
        return False
    elif 'mail' in url:
        return False
    else:
        return ((browse_boolean and (('browse' not in url) or ('page' in url))) or not browse_boolean)

In [57]:
prune('https://data.sfgov.org/City-Management-and-Ethics/Gifts-of-Travel-Contributors/hpj3-vnwh', False)

True

In [58]:
def get_all_dataset_pages(url, add_to_url, max_ds_num):
    """
    This function will get all urls ending in '/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=##'
    """
    range_of_ds = range(1,(max_ds_num + 20)//10)
    ds_pages = []
    for i in range_of_ds:
        ds_pages += [(url + add_to_url + str(i))]
    return ds_pages
    

In [59]:
get_all_dataset_pages('https://data.sfgov.org/', 'browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=', 299)

['https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=1',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=2',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=3',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=4',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=5',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=6',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=7',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=8',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=9',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=10',
 'https://data.sfgov.org/brow

In [74]:
def search(url, add_to_url = '/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=', browse_boolean = False, data_set_list = []):
    """
    This function will search through the 
    web graph in order to find the datasets
    """
    ds_len = data_amount.main(url)
    url = url + add_to_url
    url_list = []
    dataset_pages = get_all_dataset_pages(url, add_to_url, ds_len)
    pattern = re.compile(r'/([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')
    
    for url in dataset_pages:
        try:
            http_response = request.urlopen(url)
            open_response = http_response.read().decode('utf-8')
            raw_html = lxml.html.fromstring(open_response)
        except:
            continue
        for link in raw_html.xpath('//a/@href'):
            if '#' in link:
                continue
            if ('limit' in link and add_to_url == ''):
                continue
            if link[-1:] == '#':
                continue
            if link in url:
                continue
            if "http" not in link:
                link = url + link
            else:
                if pattern.match(link[-10:]) is not None and 'socrata' not in link.lower():
                    print("Made it:{}".format(link))
                    if link in data_set_list:
                        continue
                    else:
                        data_set_list += [link]
                    continue
    return data_set_list

In [75]:
ds = search('https://data.sfgov.org')
ds

Made it:https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am
Made it:https://data.sfgov.org/Public-Safety/-Change-Notice-Police-Department-Incidents/tmnf-yvry
Made it:https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis
Made it:https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6
Made it:https://data.sfgov.org/Health-and-Social-Services/HSA-90-day-emergency-shelter-waitlist/w4sk-nq57
Made it:https://data.sfgov.org/Geographic-Locations-and-Boundaries/Street-Names/6d9h-4u5v
Made it:https://data.sfgov.org/Economy-and-Community/Mobile-Food-Facility-Permit/rqzj-sfat
Made it:https://data.sfgov.org/d/jjew-r69b
Made it:https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/j2j3-acqj
Made it:https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/v3c6-n4qs
Made it:https://data.sfgov.org/Public-Safety/Fire-Department-Calls-f

Made it:https://data.sfgov.org/Energy-and-Environment/San-Francisco-Communitywide-Greenhouse-Gas-Invento/btm4-e4ak
Made it:https://data.sfgov.org/City-Infrastructure/Web-Analytics-for-SFGov-Sites-2015-Q1-Q2-Q3-Q4-/yrfc-c5yu
Made it:https://data.sfgov.org/Housing-and-Buildings/Fire-Violations/4zuq-2cbe
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-FPPC-Form-496-Late-Independent-Ex/4fd4-wqps
Made it:https://data.sfgov.org/Ethics/Campaign-Finance-Data-Key/wygs-cc76
Made it:https://data.sfgov.org/Housing-and-Buildings/Petitions-to-the-Rent-Board/6swy-cmkq
Made it:https://data.sfgov.org/City-Management-and-Ethics/Job-Titles-by-Classification/58un-vqfs
Made it:https://data.sfgov.org/Housing-and-Buildings/SF-Development-Pipeline-2014-Q4/858q-nwrm
Made it:https://data.sfgov.org/Housing-and-Buildings/Residential-Projects-With-Inclusionary-Requirement/nj3x-rw36
Made it:https://data.sfgov.org/Housing-and-Buildings/2013-Housing-Inventory/e7d3-dxh5
Made it:https://data.

Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Consultants-Client-Payments/tc9q-72uj
Made it:https://data.sfgov.org/City-Infrastructure/Wireless-Service-Facility-Permits/xtj2-daw9
Made it:https://data.sfgov.org/Housing-and-Buildings/SF-Development-Pipeline-2016-Q4/d7xx-7z6v
Made it:https://data.sfgov.org/Housing-and-Buildings/SF-Development-Pipeline-2013-Q3/hxup-t2n6
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-FPPC-Form-460-Schedule-H-Loans-Ma/62ex-d3qk
Made it:https://data.sfgov.org/Ethics/Campaign-Finance-Data-Key/wygs-cc76
Made it:https://data.sfgov.org/City-Management-and-Ethics/Citywide-Performance-Measurement-Annual-Report-FY1/5x94-tptc
Made it:https://data.sfgov.org/City-Management-and-Ethics/-alpha-Master-data-dictionary/wn8x-uk7i
Made it:https://data.sfgov.org/Housing-and-Buildings/SF-Development-Pipeline-2014-Q1/g383-7xmf
Made it:https://data.sfgov.org/City-Management-and-Ethics/Permit-Consultant-Registration-and-Quarterly-Di

Made it:https://data.sfgov.org/Housing-and-Buildings/Mayor-s-Office-of-Housing-and-Community-Developmen/2sdw-jdpk
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Summary-Of-Third-Party-Disclosure/ani6-nqh6
Made it:https://data.sfgov.org/Transportation/-Testing-SFMTA-Enforced-Temporary-Tow-Zones/6r5h-j298
Made it:https://data.sfgov.org/Transportation/Nighttime-Power-Runups/aqb8-9r2r
Made it:https://data.sfgov.org/City-Management-and-Ethics/Certificate-of-Lobbyist-Training-Filings/p7y3-jyh7
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Individual-Expenditure-Ceilings-I/wv7d-caby
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Public-Funds-Disbursed/dbak-p2fq
Made it:https://data.sfgov.org/City-Management-and-Ethics/Post-Employment-Restriction-Waivers/qwnt-35i6
Made it:https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Committee-Name-to-Measure-or-Cand/384x-35f2
Made it:https://data.sfgov.or

['https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am',
 'https://data.sfgov.org/Public-Safety/-Change-Notice-Police-Department-Incidents/tmnf-yvry',
 'https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis',
 'https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6',
 'https://data.sfgov.org/Health-and-Social-Services/HSA-90-day-emergency-shelter-waitlist/w4sk-nq57',
 'https://data.sfgov.org/Geographic-Locations-and-Boundaries/Street-Names/6d9h-4u5v',
 'https://data.sfgov.org/Economy-and-Community/Mobile-Food-Facility-Permit/rqzj-sfat',
 'https://data.sfgov.org/d/jjew-r69b',
 'https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/j2j3-acqj',
 'https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/v3c6-n4qs',
 'https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3',
 'https://data.sfgov.or

In [71]:
url = 'https://data.sfgov.org'
type(data_amount.main(url))

int

In [10]:
prog = re.compile(r'([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')
prog.match(url[-9:]) is not None

True

In [25]:
pattern = re.compile(r'/([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')


In [26]:
link = 'https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Summary-Of-Third-Party-Disclosure/7i7v-m2wn'
pattern.match('/7i7v-m2wn') is not None

True